# Prepare versions of NWM Reanalysis dataset in Zarr format

* Save a subset of NWM reanalysis covering a single HUC2 for testing purposes. The size is supposed to be not so small that it is trivial, but not so big that things take a really long time to run. 
* Rechunk this subset so we have versions with rectangular chunks, row chunks, and column chunks.|

## Setup

In [3]:
import json
from os.path import basename, join
import os 
import shutil
from os import makedirs

from dask.distributed import Client
from dask_gateway import Gateway
import xarray as xr
import fsspec
from rechunker import rechunk
import s3fs
import fsspec

%matplotlib inline

def get_json(uri):
    with fsspec.open(uri) as fd:
        return json.load(fd)

In [4]:
# Connect to existing cluster using cluster.name

# This constant needs to be set!
cluster_name = ''
gateway = Gateway()
cluster = gateway.connect(cluster_name)
client = cluster.get_client()

KeyError: 'start_time'

In [40]:
# Set various URIs.

# The CHRTOUT data from the NWM Retrospective Zarr 2.1 dataset
# This has "Streamflow values at points associated with flow lines" 
# See https://registry.opendata.aws/nwm-archive/
nwm_uri = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'

# URI of JSON file with COMIDS to use for creating subset of NWM. 
# This was produced by save_huc2_comids.ipynb.
comid_uri = 's3://azavea-noaa-hydro-data/noaa/huc2-comids.json'

# This root URI should be updated for each run of this notebook.
out_root_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/lf/07-08-2022a'

In [ ]:
nwm_subset_uri = join(out_root_uri, 'nwm-subset.zarr')
rechunk_tmp_uri = join(out_root_uri, 'rechunk-tmp.zarr')
trans_chunk_uri = join(out_root_uri, 'trans-chunk.zarr')

## Save subet of NWM

In [41]:
ds = xr.open_zarr(fsspec.get_mapper(nwm_uri, anon=True, requester_pays=True))

comids = get_json(comid_uri)['comids']

avail_comids = list(set(ds.feature_id.values).intersection(set(comids)))
# Need the following line to sort the comids or will get the following warning.
# PerformanceWarning: Slicing with an out-of-order index is generating 736 times more chunks
avail_comids.sort()
print(
    f'There are {len(comids)} reaches in the HUC and {len(avail_comids)} of those are in NWM.')

There are 137288 reaches in the HUC and 122256 of those are in NWM.


In [42]:
# time_range = slice('01-01-1990', '01-01-2000')
time_range = slice('01-01-1990', '01-01-2000')

In [43]:
ds.streamflow

<xarray.DataArray 'streamflow' (time: 367439, feature_id: 2776738)>
dask.array<open_dataset-75c8fbc6ae42016a52b62777bc4c3ce2streamflow, shape=(367439, 2776738), dtype=float64, chunksize=(672, 30000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    gage_id     (feature_id) |S15 dask.array<chunksize=(2776738,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [44]:
sub_ds = ds.sel(time=time_range, feature_id=avail_comids)

# Fails without this block. See https://github.com/pydata/xarray/issues/5219 
def remove_chunks_encoding(ds):
    for var in list(ds.keys()) + list(ds.coords):
        if 'chunks' in ds[var].encoding:
            del ds[var].encoding['chunks']
    return ds

sub_ds = remove_chunks_encoding(sub_ds)
sub_ds

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.Dataset>
Dimensions:     (feature_id: 122256, time: 87672)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(122256,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 2000-01-01T23:00:00
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(409, 445), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(409, 445), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [45]:
sub_ds.streamflow

<xarray.DataArray 'streamflow' (time: 87672, feature_id: 122256)>
dask.array<getitem, shape=(87672, 122256), dtype=float64, chunksize=(672, 24143), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(122256,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(122256,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(122256,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 2000-01-01T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [47]:
# Fails with the following error if chunk(<int>) isn't called.
# ValueError: Zarr requires uniform chunk sizes except for final chunk. Variable named 'streamflow' has incompatible dask chunks: ((48,), (445, 5348, 802, 2192, 3915, 10509, 12946, 3414, 7504, 87, 13247, 6817, 24143, 2, 11164, 1156, 3632, 499, 1518, 1666, 1633, 8, 3, 4965, 3147, 723, 771)). Consider rechunking using `chunk()`.
sub_ds = remove_chunks_encoding(sub_ds.chunk({'time': 672, 'feature_id': 30000}))
# sub_ds = remove_chunks_encoding(sub_ds.chunk(1000))

In [50]:
sub_ds.streamflow

<xarray.DataArray 'streamflow' (time: 87672, feature_id: 122256)>
dask.array<rechunk-merge, shape=(87672, 122256), dtype=float64, chunksize=(672, 30000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(30000,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(30000,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(30000,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(30000,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(30000,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 2000-01-01T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [51]:
%%time 

sub_ds.to_zarr(nwm_subset_uri)

CPU times: user 9.63 s, sys: 409 ms, total: 10 s
Wall time: 9min 55s


In [52]:
saved_ds = xr.open_zarr(nwm_subset_uri)
saved_ds.streamflow

<xarray.DataArray 'streamflow' (time: 87672, feature_id: 122256)>
dask.array<open_dataset-8b0d3d732749ab84324bc6d080f3f780streamflow, shape=(87672, 122256), dtype=float64, chunksize=(672, 30000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(30000,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(30000,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(30000,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(30000,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(30000,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 2000-01-01T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [14]:
sub_ds.chunks

Frozen({'feature_id': (1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 256), 'time': (48,)})

## Save rechunked versions of subset of NWM.

In [54]:
def _rechunk(target_chunks, output_uri):
    max_mem = '2GB'
    rm(rechunk_tmp_uri)
    ds = xr.open_zarr(nwm_subset_uri)

    # Note, if you get a ContainsArrayError, you probably need to delete temp_store and target_store first.
    # See https://github.com/pangeo-data/rechunker/issues/78

    targ_store = fsspec.get_mapper(output_uri)
    temp_store = fsspec.get_mapper(rechunk_tmp_uri)
    rechunk_plan = rechunk(ds, target_chunks, max_mem, targ_store, temp_store=temp_store)
    rechunk_plan.execute()

In [55]:
%%time

# Each chunk has dimension that is the transpose of the original chunk size.
trans_chunk_uri = join(out_root_uri, 'trans-chunk.zarr')
target_chunks = {'time': 30000, 'feature_id': 672} 
_rechunk(target_chunks, trans_chunk_uri)

CPU times: user 938 ms, sys: 57.7 ms, total: 995 ms
Wall time: 6min 45s


In [56]:
rechunked_ds = xr.open_zarr(square_chunk_uri)
rechunked_ds.chunks

/tmp/ipykernel_29/3337888933.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  rechunked_ds = xr.open_zarr(square_chunk_uri)


Frozen({'feature_id': (672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 624), 'time': (30000, 30000, 27672)})